In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
import timm
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


CONFIG = {
    'image_dir': '/kaggle/input/notebooks/kagglertw/shemagh-right-placev2/images',
    'csv_path': '/kaggle/input/notebooks/kagglertw/shemagh-right-placev2/train_df.csv',
    'img_size': 256,  
    'batch_size': 24,  
    'num_epochs': 30,
    'learning_rate': 3e-5,  
    'weight_decay': 1e-3,  
    'num_folds': 5,
    'seed': 42,
    'num_workers': 2,
    'model_name': 'convnext_tiny.fb_in22k_ft_in1k',  
    'save_dir': './weights_convnext',
    'early_stopping_patience': 7,  
    'use_class_weights': True,
    'dropout': 0.3,
    'label_smoothing': 0.1,  
    'use_mixup': True,  
    'mixup_alpha': 0.2,
}

os.makedirs(CONFIG['save_dir'], exist_ok=True)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CONFIG['seed'])


def mixup_data(x, y, alpha=0.2):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


class BinaryImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'filename']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.df.loc[idx, 'right_place']
        return image, torch.tensor(label, dtype=torch.float32)


train_transform = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.TrivialAugmentWide(),  
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.2)),
])

val_transform = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])


class ConvNeXtClassifier(nn.Module):
    def __init__(self, model_name='convnext_tiny.fb_in22k_ft_in1k', pretrained=True, dropout=0.3):
        super(ConvNeXtClassifier, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=0)
        in_features = self.model.num_features
        
        
        self.head = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Dropout(dropout),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout / 2),
            nn.Linear(512, 1)
        )
        
    def forward(self, x):
        features = self.model(x)
        return self.head(features).squeeze()


class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.001, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        
    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif self.mode == 'max':
            if score < self.best_score + self.min_delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.counter = 0
        elif self.mode == 'min':
            if score > self.best_score - self.min_delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.counter = 0

def train_epoch(model, loader, criterion, optimizer, device, use_mixup=False, mixup_alpha=0.2, label_smoothing=0.0):
    model.train()
    running_loss = 0.0
    predictions = []
    targets = []
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        
        if label_smoothing > 0:
            labels_smooth = labels * (1 - label_smoothing) + 0.5 * label_smoothing
        else:
            labels_smooth = labels
        
        if use_mixup and np.random.rand() > 0.5:
            images, labels_a, labels_b, lam = mixup_data(images, labels_smooth, mixup_alpha)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
            loss.backward()
            optimizer.step()
            
            
            preds = torch.sigmoid(outputs).detach().cpu().numpy()
            predictions.extend(preds)
            targets.extend(labels.cpu().numpy())
        else:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels_smooth)
            loss.backward()
            optimizer.step()
            
            preds = torch.sigmoid(outputs).detach().cpu().numpy()
            predictions.extend(preds)
            targets.extend(labels.cpu().numpy())
        
        running_loss += loss.item() * images.size(0)
        pbar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss, np.array(predictions), np.array(targets)

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    predictions = []
    targets = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            preds = torch.sigmoid(outputs).cpu().numpy()
            predictions.extend(preds)
            targets.extend(labels.cpu().numpy())
            
            pbar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss, np.array(predictions), np.array(targets)

def calculate_metrics(predictions, targets, threshold=0.5):
    pred_binary = (predictions >= threshold).astype(int)
    accuracy = accuracy_score(targets, pred_binary)
    auc = roc_auc_score(targets, predictions)
    f1 = f1_score(targets, pred_binary)
    precision = precision_score(targets, pred_binary, zero_division=0)
    recall = recall_score(targets, pred_binary, zero_division=0)
    return accuracy, auc, f1, precision, recall

def train_fold(fold, train_df, val_df, config):
    print(f"\n{'='*60}")
    print(f"Training Fold {fold + 1}/{config['num_folds']} - ConvNeXt-Tiny")
    print(f"{'='*60}")
    
    
    train_dataset = BinaryImageDataset(train_df, config['image_dir'], train_transform)
    val_dataset = BinaryImageDataset(val_df, config['image_dir'], val_transform)
    
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], 
                            shuffle=True, num_workers=config['num_workers'], pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], 
                          shuffle=False, num_workers=config['num_workers'], pin_memory=True)
    
    
    model = ConvNeXtClassifier(
        model_name=config['model_name'],
        pretrained=True, 
        dropout=config['dropout']
    ).to(device)
    
    
    if config['use_class_weights']:
        pos_count = (train_df['right_place'] == 1).sum()
        neg_count = (train_df['right_place'] == 0).sum()
        pos_weight = torch.tensor([neg_count / pos_count]).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        print(f"Pos weight: {pos_weight.item():.2f}")
    else:
        criterion = nn.BCEWithLogitsLoss()
    
    
    label_smoothing = config['label_smoothing']
    
    
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    
    
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=config['learning_rate'] * 3,
        epochs=config['num_epochs'],
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        anneal_strategy='cos'
    )
    
    
    early_stopping = EarlyStopping(
        patience=config['early_stopping_patience'],
        mode='max'
    )
    
    best_val_loss = float('inf')
    best_auc = 0.0
    best_metrics = {}
    
    
    for epoch in range(config['num_epochs']):
        current_lr = optimizer.param_groups[0]['lr']
        print(f"\nEpoch {epoch + 1}/{config['num_epochs']} | LR: {current_lr:.2e}")
        
        
        train_loss, train_preds, train_targets = train_epoch(
            model, train_loader, criterion, optimizer, device,
            use_mixup=config['use_mixup'],
            mixup_alpha=config['mixup_alpha'],
            label_smoothing=label_smoothing
        )
        train_acc, train_auc, train_f1, train_prec, train_rec = calculate_metrics(
            train_preds, train_targets
        )
        
        
        val_loss, val_preds, val_targets = validate_epoch(
            model, val_loader, criterion, device
        )
        val_acc, val_auc, val_f1, val_prec, val_rec = calculate_metrics(
            val_preds, val_targets
        )
        
        print(f"Train | Loss: {train_loss:.4f} | AUC: {train_auc:.4f} | F1: {train_f1:.4f} | Acc: {train_acc:.4f}")
        print(f"Val   | Loss: {val_loss:.4f} | AUC: {val_auc:.4f} | F1: {val_f1:.4f} | Acc: {val_acc:.4f}")
        print(f"Val   | Precision: {val_prec:.4f} | Recall: {val_rec:.4f}")
        
        
        if val_auc > best_auc:
            best_auc = val_auc
            best_val_loss = val_loss
            best_metrics = {
                'epoch': epoch,
                'val_loss': val_loss,
                'val_auc': val_auc,
                'val_acc': val_acc,
                'val_f1': val_f1,
                'val_precision': val_prec,
                'val_recall': val_rec,
            }
            torch.save({
                **best_metrics,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'config': config,
            }, os.path.join(config['save_dir'], f'convnext_fold{fold+1}_best.pth'))
            print(f"✓ Saved best model (AUC: {best_auc:.4f}, F1: {val_f1:.4f})")
        
        
        early_stopping(val_auc)
        if early_stopping.early_stop:
            print(f"⚠ Early stopping at epoch {epoch + 1}")
            break
    
    return best_val_loss, best_auc, best_metrics

def main():
    print(f"\n{'='*60}")
    print("ConvNeXt-Tiny Binary Classifier with Advanced Techniques")
    print(f"{'='*60}\n")
    
    print("Loading data...")
    df = pd.read_csv(CONFIG['csv_path'])
    print(f"Dataset: {df.shape[0]} samples, {df.shape[1]} columns")
    print(f"\nClass distribution:")
    class_dist = df['right_place'].value_counts()
    print(class_dist)
    imbalance_ratio = class_dist[0] / class_dist[1]
    print(f"Imbalance ratio: {imbalance_ratio:.2f}:1")
    
    
    skf = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    
    fold_results = []
    
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['right_place'])):
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
        
        print(f"\n{'='*60}")
        print(f"Fold {fold + 1}/{CONFIG['num_folds']}")
        print(f"{'='*60}")
        print(f"Train: {len(train_df)} samples | Val: {len(val_df)} samples")
        print(f"Train dist: {dict(train_df['right_place'].value_counts())}")
        print(f"Val dist: {dict(val_df['right_place'].value_counts())}")
        
        val_loss, val_auc, metrics = train_fold(fold, train_df, val_df, CONFIG)
        fold_results.append({
            'fold': fold + 1, 
            'val_loss': val_loss, 
            'val_auc': val_auc,
            'val_f1': metrics['val_f1'],
            'val_acc': metrics['val_acc'],
            'val_precision': metrics['val_precision'],
            'val_recall': metrics['val_recall'],
        })
        
        
        torch.cuda.empty_cache()
    
    
    print(f"\n{'='*60}")
    print("CROSS-VALIDATION SUMMARY - ConvNeXt-Tiny")
    print(f"{'='*60}")
    results_df = pd.DataFrame(fold_results)
    print(results_df.to_string(index=False))
    print(f"\n{'='*60}")
    print(f"Mean AUC:       {results_df['val_auc'].mean():.4f} ± {results_df['val_auc'].std():.4f}")
    print(f"Mean F1:        {results_df['val_f1'].mean():.4f} ± {results_df['val_f1'].std():.4f}")
    print(f"Mean Accuracy:  {results_df['val_acc'].mean():.4f} ± {results_df['val_acc'].std():.4f}")
    print(f"Mean Precision: {results_df['val_precision'].mean():.4f} ± {results_df['val_precision'].std():.4f}")
    print(f"Mean Recall:    {results_df['val_recall'].mean():.4f} ± {results_df['val_recall'].std():.4f}")
    print(f"Mean Loss:      {results_df['val_loss'].mean():.4f} ± {results_df['val_loss'].std():.4f}")
    print(f"{'='*60}\n")
    
    
    results_df.to_csv(os.path.join(CONFIG['save_dir'], 'cv_results.csv'), index=False)
    print(f"✓ Results saved to {CONFIG['save_dir']}/cv_results.csv")
    print(f"✓ Model weights saved in {CONFIG['save_dir']}/")

if __name__ == '__main__':
    main()

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Using device: cuda

ConvNeXt-Tiny Binary Classifier with Advanced Techniques

Loading data...
Dataset: 833 samples, 2 columns

Class distribution:
right_place
0    646
1    187
Name: count, dtype: int64
Imbalance ratio: 3.45:1

Fold 1/5
Train: 666 samples | Val: 167 samples
Train dist: {0: np.int64(516), 1: np.int64(150)}
Val dist: {0: np.int64(130), 1: np.int64(37)}

Training Fold 1/5 - ConvNeXt-Tiny


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Pos weight: 3.44

Epoch 1/30 | LR: 3.60e-06


Validation: 100%|██████████| 7/7 [00:03<00:00,  2.16it/s, loss=1.46]


...

Train | Loss: 0.8779 | AUC: 0.7914 | F1: 0.5430 | Acc: 0.6897
Val   | Loss: 0.8370 | AUC: 0.7964 | F1: 0.5854 | Acc: 0.6928
Val   | Precision: 0.4186 | Recall: 0.9730

Epoch 11/30 | LR: 3.60e-06


Validation: 100%|██████████| 7/7 [00:01<00:00,  6.13it/s, loss=0.881]


Train | Loss: 0.8579 | AUC: 0.7743 | F1: 0.5525 | Acc: 0.7061
Val   | Loss: 0.8403 | AUC: 0.7939 | F1: 0.6000 | Acc: 0.7108
Val   | Precision: 0.4337 | Recall: 0.9730

Epoch 12/30 | LR: 3.60e-06


Validation: 100%|██████████| 7/7 [00:01<00:00,  6.46it/s, loss=0.878]


Train | Loss: 0.8548 | AUC: 0.8029 | F1: 0.5885 | Acc: 0.7316
Val   | Loss: 0.8385 | AUC: 0.7915 | F1: 0.6050 | Acc: 0.7169
Val   | Precision: 0.4390 | Recall: 0.9730
⚠ Early stopping at epoch 12

CROSS-VALIDATION SUMMARY - ConvNeXt-Tiny
 fold  val_loss  val_auc   val_f1  val_acc  val_precision  val_recall
    1  0.843797 0.841372 0.585366 0.694611       0.418605    0.972973
    2  0.703025 0.881579 0.660714 0.772455       0.500000    0.973684
    3  0.706919 0.860363 0.686275 0.808383       0.546875    0.921053
    4  0.729289 0.887597 0.654206 0.777108       0.500000    0.945946
    5  0.865956 0.799811 0.576000 0.680723       0.409091    0.972973

Mean AUC:       0.8541 ± 0.0354
Mean F1:        0.6325 ± 0.0489
Mean Accuracy:  0.7467 ± 0.0558
Mean Precision: 0.4749 ± 0.0590
Mean Recall:    0.9573 ± 0.0235
Mean Loss:      0.7698 ± 0.0787

✓ Results saved to ./weights_convnext/cv_results.csv
✓ Model weights saved in ./weights_convnext/
